In [1]:
from aitk.robots import World, Scribbler
from aitk.robots.utils import distance
from aitk.networks import Network
from aitk.algorithms import GeneticAlgorithm
from aitk.utils import Color

import numpy as np
import random
from tensorflow.keras.layers import Input, Dense

In [2]:
world = World(100, 100)
world.add_wall("blue", 0, 30, 65, 35)
world.add_wall("blue", 0, 70, 65, 65)
world.add_wall("blue", 65, 35, 70, 0)
world.add_wall("blue", 65, 65, 70, 100)
robot = Scribbler(10, 50, 0)
robot.add_device_ring("RangeSensor", 0, -180, 180, 16, 
                      width=0, max=100)
world.add_robot(robot)
world.add_bulb("yellow", 85, 50, 0, 20)
world.watch()

Random seed set to: 3591189


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [3]:
def get_sensor_array(robot):
    return [x.get_reading() for x in robot if x.type == "laser"]

In [17]:
print(get_sensor_array(robot))

[0.19306966449366622, 0.20226044250321185, 0.20799078716367408, 0.15145643723984623, 0.1352807956859265, 0.14172062539902386, 0.17711632458393345, 0.2961800554299056, 0.8101388911068693, 0.8487042800538572, 0.2547315144994178, 0.1854924834167377, 0.16568177099423412, 0.17356879136809686, 0.21691878865928726, 0.21615517099109713]


In [4]:
sensor_array = get_sensor_array(robot)
len(sensor_array)

16

In [5]:
def create_model(robot):
    sensor_array = get_sensor_array(robot)
    network = Network(layers=[
        Input(1, name="signal"),
        Input(len(sensor_array), name="sensors"),
        Dense(10, name="hidden", activation="sigmoid"),
        Dense(2, name="output", activation="sigmoid"),
    ])
    network.connect("signal", "hidden")
    network.connect("sensors", "hidden")
    network.connect("hidden", "output")
    network.compile()
    return network

In [6]:
network = create_model(robot)

Connect layers with Network.connect(NAME, NAME) where NAMEs are in:
     ['signal', 'sensors', 'hidden', 'output']


In [7]:
network.propagate(
        [[1],  get_sensor_array(robot)],
) * 2 - 1

array([-0.04840964,  0.51975214], dtype=float32)

In [8]:
network.watch()

HTML(value='<div style="outline: 5px solid #1976D2FF; width: 400px; height: 209px;"><svg id=\'keras-network\' …

In [9]:
network.model.summary()

Model: "Network"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
signal (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
sensors (InputLayer)            [(None, 16)]         0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 17)           0           signal[0][0]                     
                                                                 sensors[0][0]                    
__________________________________________________________________________________________________
hidden (Dense)                  (None, 10)           180         concatenate[0][0]          

In [10]:
len(network.get_weights(flat=True))

202

In [11]:
class GA(GeneticAlgorithm):
    def __init__(self, world, network, popSize, **kwargs):
        self.world = world
        self.network = network
        self.length = len(self.network.get_weights(flat=True))
        self.black = Color("black")
        self.white = Color("white")
        super().__init__(self.length, popSize, **kwargs)

    def controller(self, robot):
        if robot.stalled:
            return True
        signal = np.array([robot.state["signal"]])
        sensors = get_sensor_array(robot)
        output = self.network.propagate(
            [signal, sensors],
            show=robot.state["show"]) * 2 - 1
        robot.move(*output)
        if robot.state["count"] >= 10:
            robot.state["signal"] = 0.0
            robot.world.bulbs[0].off()
        elif robot.state["count"] == 0:
            robot.world.bulbs[0].on()
            if robot.state["signal"] == 1:
                robot.world.bulbs[0].set_color("yellow")
            else: 
                robot.world.bulbs[0].set_color("orange")
        robot.state["count"] += 1
        
    def is_done(self):
        return self.bestEverScore > 495
        
    def make_random_gene(self):
        return 1 - random.random() * 2
    
    def mutate_gene(self, gene):
        return gene + 0.05 - random.random() * 0.10

    def fitness(self, chromosome, seconds=8.0, real_time=False):
        robot = world.robots[0]
        robot.state["show"] = real_time
        self.network.set_weights(chromosome)
        total = 0
        for signal, target in [(1, (85, 10)), 
                                (-1, (85, 90))]:
            robot.state["signal"] = signal
            robot.state["count"] = 0
            robot.set_pose(10, 50, 0)
            robot.world.canvas.clear()
            for color, radius in [(self.black, 6), (self.white, 4), (self.black, 2)]:
                robot.world.canvas.set_stroke_style(color, 1)
                robot.world.canvas.set_fill_style(color)
                robot.world.canvas.draw_circle(
                    target[0], target[1], radius)
            self.world.seconds(
                seconds, 
                [self.controller], 
                real_time=real_time,
                show=True,
                show_progress=real_time, 
                interrupt=True,
                quiet=True)
            total += distance(robot.x, robot.y, 
                              target[0], target[1])
        return 500 - total


In [12]:
ga = GA(world, network, 50)

Genetic algorithm
  Chromosome length: 202
  Population size: 50


In [13]:
ga.watch()

HTML(value='<?xml version="1.0" encoding="utf-8" standalone="no"?>\n<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1/…

In [14]:
ga.fitness(ga.make_random_chromosome(), real_time=True)

327.8964074524648

In [15]:
best = ga.evolve(300, elite_percent=.020, mutation_rate=0.01)

Maximum number of generations: 300
  Elite percentage 0.02 (1/50 chromosomes per generation)
  Crossover rate: 0.7 (~34/49 crossovers per generation)
  Mutation rate: 0.01 (~197/9898 genes per generation)
Evaluating initial population...
Done!


  0%|          | 0/300 [00:00<?, ?it/s]

Manually interrupted


In [18]:
ga.fitness(best, real_time=True)

415.04848625871335

In [21]:
saved_weights = [
    0.02, 0.76, -0.51, -0.68, -0.82, 0.86, -0.96, 0.45, -0.5, 
    0.78, 0.34, -0.27, -0.88, -1.01, -0.21, 0.34, -0.05, 0.48, 
    -0.84, 0.38, -0.57, 0.14, -0.65, 0.12, -0.68, -0.79, 0.22, 
    0.07, 0.17, -0.46, 0.3, -1.0, -0.65, -0.02, -0.65, 0.94, 
    -0.74, 0.53, 0.62, 0.16, 0.65, -0.28, 0.42, -0.12, 0.66, 
    0.58, 0.91, 0.02, 0.2, 0.64, -0.75, -0.15, -0.42, -0.71, 
    0.16, -0.88, 0.89, -0.16, 0.45, 0.27, -0.16, 0.73, 0.38, 
    -0.42, 0.32, 0.49, 0.23, 1.0, -0.08, -0.31, -0.45, 0.12, 
    -0.85, 0.07, -0.47, -0.11, 0.81, 0.47, 0.61, -0.09, 0.95, 
    1.01, -0.96, 0.24, -0.89, 0.07, 0.79, -0.4, 0.71, 0.29, 
    0.22, 0.29, -0.39, 0.86, 0.73, -0.77, -0.31, 0.1, -0.35, 
    0.59, -0.66, -0.3, 0.54, -0.89, -0.07, -0.25, 0.69, -0.32, 
    -0.27, 0.48, 0.04, -0.05, 0.43, -0.14, 0.92, 0.25, -0.59, 
    0.77, -0.14, -0.21, 0.22, -0.68, 1.02, -0.12, 0.08, -0.13, 
    -0.07, 0.01, 0.45, 0.85, 0.35, 0.22, 0.73, 0.39, 0.56, 0.06, 
    0.49, 0.33, 0.96, 0.24, 0.93, 0.77, 0.11, -0.03, -0.96, 0.95, 
    0.52, -1.05, -0.53, 0.64, -0.63, -0.2, -0.78, 0.03, -0.92, 
    0.36, 0.52, -0.14, -0.96, 0.52, -0.43, -0.41, -0.84, -0.01, 
    0.83, 0.06, -0.87, -0.96, -0.55, -0.45, -0.47, -0.7, 0.84, 
    -0.55, -0.83, 0.76, 0.75, 0.76, 0.55, -0.19, 0.37, 0.45, 
    0.95, -0.94, -0.35, 0.84, 0.87, 0.8, -0.21, 0.82, 0.48, 0.35, 
    0.51, 0.12, -0.44, -0.42, 0.03, -0.41, 1.0, 0.84, -0.88, -0.97
]

In [22]:
ga.fitness(saved_weights, seconds=15, real_time=True)

497.3044822769719